# formata-texto
Esse script recebe arquivos .txt extraídos do site da Câmara dos Deputados e executa operações de limpeza e formatação usando regex.

#### Importação de pacotes

In [1]:
import glob
import os
import pandas as pd
import re
from unidecode import unidecode

In [2]:
# VARIÁVEL GLOBAL PARA TESTAR VÁRIOS ENCODINGS
encoding = 'UTF-8'

#### Ler arquivos externos

In [3]:
def find_fpaths(dir_path, pattern):
    '''
    >> DESCRIÇÃO
    
    Usa o módulo glob para buscar todos os arquivos
    que correspondam ao padrão passado na variável 
    pattern'. Retorna uma lista de paths no formato 
    string. 
    
    >> PARÂMETROS
    
    dir_path -> uma string com o caminho para o
    diretório onde a busca pelos arquivos será
    realizada.
    
    pattern -> uma string com o padrão de texto
    que deve ser procurado no diretório.
    
    '''
    
    full_pattern = dir_path + pattern
    files = glob.glob(full_pattern)
    
    return files

In [4]:
def make_df_plen(file_list):
    '''
    >> DESCRIÇÃO
    
    Lê a lista de arquivos e configura o conteúdo
    em um dataframe com os seguintes campos:
    PRESIDENTE | CONTEUDO | ARQUIVO | ANO
    Funciona para os discursos em plenário
    
    >> PARÂMETROS
    
    file_list -> uma lista de filepaths em formato
    string. Ela é gerada anteriormente, na função
    find_fpaths.
    
    '''
    
    # Lê o conteúdo dos arquivos de texto na lista
    content = [ open(file, encoding = encoding).read() for file in file_list ]
    file_list = [ unidecode(file) for file in file_list ]
    
    # Transforma eum dataframe
    df = pd.DataFrame({
        'FILE'             : [ file for file in file_list ],
        'ORIGINAL_CONTENT' : [ item for item in content ],
        'CLEAN_CONTENT'    : [ unidecode(item) for item in content ],
        'SESSION_TYPE'     : [ re.search("\-([A-Z\s]+)\-", file).group(1) for file in file_list ],
        'SESSION_DATE'     : [ re.search("\d{8}", file).group(0) for file in file_list ]
    })
    
    df["SESSION_DATE"] = pd.to_datetime(df.SESSION_DATE, format = "%d%m%Y")

    return df

In [5]:
def make_df_com(file_list):
    '''
    >> DESCRIÇÃO
    
    Lê a lista de arquivos e configura o conteúdo
    em um dataframe com os seguintes campos:
    PRESIDENTE | CONTEUDO | ARQUIVO | ANO
    Funciona para os discursos em comissão
    
    >> PARÂMETROS
    
    file_list -> uma lista de filepaths em formato
    string. Ela é gerada anteriormente, na função
    find_fpaths.
    
    '''
    
    # Lê o conteúdo dos arquivos de texto na lista
    content = [ open(file, encoding = encoding).read() for file in file_list ]
    file_list = [ unidecode(file) for file in file_list ]
    
    # Transforma eum dataframe
    df = pd.DataFrame({
        'FILE'             : [ file for file in file_list ],
        'ORIGINAL_CONTENT' : [ item for item in content ],
        'CLEAN_CONTENT'    : [ unidecode(item) for item in content ],
        'SESSION_TYPE'     : [ 'COMISSÃO ESPECIAL (FORA DO PLENÁRIO)' for file in file_list ],
        'SESSION_DATE'     : [ re.search("\d{8}", file).group(0) for file in file_list ]
    })
    
    df["SESSION_DATE"] = pd.to_datetime(df.SESSION_DATE, format = "%d%m%Y")

    return df

#### Funções de formatação e busca usando regex

In [6]:
def find_speakers(string):
    
    '''
    Essa função detecta o padrão de texto
    que antecede a fala de um deputado e
    retorna um objeto match (via re.find_all).
    Ele é útil para detectar QUANTOS deputados
    falaram em determinada string textual.
    '''
    
    pattern = "((O?\s?SR\.?\s+?)|(A?\s?SRA\.?\s+?))(\s+DEPUTADO|\s+DEPUTADA|\s+PRESIDENTE|\s+PRESIDENTA)?"
    regexp = re.compile(pattern)
    matches = re.findall(regexp, string)
    
    return matches

In [7]:
def extract_full_quote(clean_string, original_string):
    
    '''
    Essa função extrai todas as falas de Jair Bolsonaro
    em uma determinada string. O pattern de regex encontra,
    primeiro, uma fala qualquer do Presidente. Então, pega
    tudo que está entre essa fala e a fala de outro parlamentar 
    ou o fim do arquivo. Isso é necessário porque há arquivos
    que misturam a fala de vários parlamentares, geralmente
    quando estão envolvidos em uma discussão.
    
    Para fazer essa operação, são passados textos sem caracteres
    especiais unicode. Depois de feita a captura, usamos os índices
    das matches no regex para extrair o mesmo pedaço de texto
    na string original.

    '''
    
    if "O SR. PRESIDENTE (Jair Bolsonaro)" in clean_string:
        pattern = "O SR\. PRESIDENTE (\(Jair Bolsonaro\))?(.*?)((O?\s?SR\.?\s+?)|(A?\s?SRA\.?\s+?)|$)"
        group_no = 2

    else:
        pattern  = "O?\s?SR\.?\s?(DEPUTADO)?\s+JAIR\s+BOLSONARO\s?(\((Bloco\/)?\w{2,}\s?\-\s?\w{2}[^)]+\))?(.*?)((O?\s?SR\.?\s+?)|(A?\s?SRA\.?\s+?)|$)"
        group_no = 4
        
    regexp   = re.compile(pattern, re.MULTILINE)
    matches  = re.finditer(regexp, clean_string)
    
    full_clean_quote    = [ ]
    full_original_quote = [ ]
    
    for match in matches:
                        
        match_start = match.start(group_no)
        match_end   = match.end(group_no)
            
        clean_quote = match[group_no]
        clean_quote = clean_quote.replace("- ", "")
        
        original_quote = original_string[match_start:match_end]
        original_quote = original_quote.replace("- ", "")
        
        full_clean_quote.append(clean_quote)
        full_original_quote.append(original_quote)
        
    full_clean_quote    = ' [ INTERRUPÇÃO ] '.join(full_clean_quote)
    full_original_quote = ' [ INTERRUPÇÃO ] '.join(full_original_quote)
    
#     # Limpa caractere maluco que aparece no começo de alguns textos
#     if full_original_quote.startswith('â'):
#         full_original_quote = full_original_quote[3:] # Esse caractere é sempre seguido de um mais dois lixinhos
    
#     if full_clean_quote.startswith('a'):
#         full_clean_quote = full_clean_quote[1:] # Os caracteres posteriores já foram obliterados pelo unidecode
        
    # Remove espaços múltiplos internos usando a operação join de lista
    full_clean_quote    = ' '.join(full_clean_quote.split()) 
    full_original_quote = ' '.join(full_original_quote.split()) 
    
#     print(f"FULL_CLEAN_QUOTE:\n{full_clean_quote}\n")
#     print(f"FULL_ORIGINAL_QUOTE:\n{full_original_quote}\n")
    
    return full_clean_quote, full_original_quote

#### Funções para aplicar operações de regex no dataframe

In [8]:
def apply_count_speakers(row):
    
    '''
    Aplica, linha a linha, a função
    find_speakers(string)
    '''
    
    matches = find_speakers(row.CLEAN_CONTENT)
    speaker_count = len(matches)
    
    return pd.Series({ "SPEAKER_COUNT":speaker_count 
        })

In [9]:
def apply_extract_full_quote(row):
    
    '''
    Aplica, linha a linha, a função
    extract_full_quote(string)
    '''
    
    full_clean_quote, full_original_quote = extract_full_quote(row.CLEAN_CONTENT, row.ORIGINAL_CONTENT)
    return pd.Series({
        "FULL_CLEAN_QUOTE"    : full_clean_quote,
        "FULL_ORIGINAL_QUOTE" : full_original_quote
    })

#### Função que encapsula anteriores e roda a operação

In [12]:
def run_script():
    
    '''
    Executa as operações anteriores em ambos os bancos de dados
    (plenário e comissões), filtra entradas sem match, concatena
    ambos os dataframes e salva para arquivo csv.
    '''
    
    # Monta dataframes
    jb_plen = make_df_plen( find_fpaths("../data/txts/plenario/", "*.txt") )
    jb_com  = make_df_com( find_fpaths("../data/txts/comissao/", "*.txt") )
    
    # As sessões do tipo HOMENAGEM são apenas registro de protocolo. 
    # Não contém transcrição de discursos.
    jb_plen = jb_plen[ jb_plen.SESSION_TYPE != "HOMENAGEM" ]
    
    # Aplica funções para extrair discursos
    jb_plen["SPEAKER_COUNT"] = jb_plen.apply(apply_count_speakers, axis=1)
    jb_plen = jb_plen.join(jb_plen.apply(apply_extract_full_quote, axis=1))
    
#     jb_com["SPEAKER_COUNT"]  = jb_com.apply(apply_count_speakers, axis=1)
#     jb_com = jb_com.join(jb_com.apply(apply_extract_full_quote, axis=1))
    
    # Há sim entradas que não tiveram match algum
    # Verifiquei manualmente e elas são de discursos
    # que o site da Câmara retornou erroneamente.
    # De fato não contém falas de Bolsonaro.
    
    jb_plen = jb_plen[ jb_plen.FULL_CLEAN_QUOTE != "" ]
#     jb_com  = jb_com[ jb_com.FULL_CLEAN_QUOTE != "" ]
    
#     jb = pd.concat( [ jb_plen, jb_com ] )
    
    # Salva
    directory = "../data/csvs/"
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    fname = directory + "discursos_jair_bolsonaro.csv" 
    jb_plen.to_csv(fname, index = False)
#    jb.to_csv(fname, index = False)
    
    return jb_plen
#    return jb

In [13]:
df = run_script()